In [20]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


class BsOption:
    def __init__(self, S, K, T, r, sigma, q=0):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.q = q

    @staticmethod
    def N(x):
        return norm.cdf(x)

    @property
    def params(self):
        return {'S': self.S,
                'K': self.K,
                'T': self.T,
                'r': self.r,
                'q': self.q,
                'sigma': self.sigma}

    def d1(self):
        return (np.log(self.S / self.K) + (self.r - self.q + self.sigma ** 2 / 2) * self.T) \
               / (self.sigma * np.sqrt(self.T))

    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)

    def _call_value(self):
        return self.S * np.exp(-self.q * self.T) * self.N(self.d1()) - \
               self.K * np.exp(-self.r * self.T) * self.N(self.d2())

    def _put_value(self):
        return self.K * np.exp(-self.r * self.T) * self.N(-self.d2()) - \
               self.S * np.exp(-self.q * self.T) * self.N(-self.d1())

    def price(self):
        # return self._call_value()
        return self._call_value(), self._put_value()


def GetData(stk, start_date, end_date):
    df = yf.download(stk, start_date, end_date, interval='1d').dropna()
    return (df)


def AddTime(df):
    df['start']=df.index
    df['end']=df.index
    df['dow'] = df.index.dayofweek
    #df['moy']
    df['woy'] = df.index.isocalendar().week + df.index.isocalendar().year * 100
    df['moy']=df.index.strftime('%m-%Y')
    return (df)

def GroupW(df,div,type):
    #df=df[df['dow']!=3]
    #df=df[df['dow']!=2]
    #df=df[df['dow']!=1]
    #df=df[df['dow']!=0]
    if type=="week":
        df['woy']=(df.woy/div).apply(np.ceil)
        df = df.groupby(['woy']).agg({'start':'first','end':'last','Open': 'first', 'Close': 'last'})
        df['days']= (df['end'] - df['start']).dt.days+1
        df=df.set_index('start')
    #if type=="month":
    #    df['moy'] = (df.moy/div).apply(np.ceil)
    #    df = df.groupby(['moy']).agg({'Open': 'first', 'Close': 'last'})
    return (df)

def Outs(df,IV):
    df['K'] = df.Open.round(0)
    df['C1'] = BsOption(df.Open, df.K, df.days / 365, 0.03, IV).price()[0]
    df['C2'] = BsOption(df.Close, df.K, 0 / 365, 0.03, IV).price()[0]
    df['P1'] = BsOption(df.Open, df.K, df.days / 365, 0.03, IV).price()[1]
    df['P2'] = BsOption(df.Close, df.K, 0 / 365, 0.03, IV).price()[1]
    df['T1'] = df.C1 + df.P1
    df['T2'] = df.C2 + df.P2
    df['PL'] = df.T1 - df.T2
    df['LogR'] = np.log((df.Open + df.PL) / df.Open)
    df['OptionYield']=df.T1/df.K
    return(df)

df0=pd.read_csv('option_data_2022-08-19_2022-08-05.csv')
#Inputs
#intervals=[1,2,4,12,24,48]
#stock=df0.symbol[0]
#start_date='2020-01-13'
#end_date='2020-12-24'

#start_date='2021-01-11'
#end_date='2021-12-24'

#start_date='2022-01-03'
#end_date='2022-06-25'

start_date='2021-01-01'
end_date='2022-08-05'

#IV = (df0.atm_call_IV[0]+df0.atm_put_IV[0])/2
div=2
type="week"
arr=[]
for i in range (0,len(df0.symbol)):
    stock=df0.symbol[i]
    IV = (df0.atm_call_IV[i]+df0.atm_put_IV[i])/2
    df = GetData(stock, start_date, end_date)
    df = AddTime(df)
    df=GroupW(df,div,type)
    df=Outs(df,IV)
    log_r=np.sum(df.LogR)
    pos_log=np.sum(df[df.LogR > 0].LogR)
    neg_log=np.sum(df[df.LogR <= 0].LogR)
    pos_periods=len(df[df.LogR > 0].LogR)
    ttl_periods=(len(df.LogR))
    avg_yield=np.average(df.LogR.dropna())
    opt_yield=np.average(df.OptionYield)
    arr.append([stock,log_r,pos_log,neg_log,pos_periods,ttl_periods,avg_yield,opt_yield])
    #print(df)
    #filename=(stock+", "+start_date+", "+end_date+", "+str(div)+".csv")
    #df.to_csv(filename)
    #Sum Log Returns
    #print(np.sum(df.LogR))
    #Sum Positive Log
    #print(np.sum(df[df.LogR > 0].LogR))
    #Sum Negative Log
    #print(np.sum(df[df.LogR <= 0].LogR))
    #Count of Positive Periods
    #print(len(df[df.LogR > 0].LogR))
    #Total Count of Periods
    #print(len(df.LogR))
    #Average Option Yield
    #print(np.average(df.OptionYield))
    #print((np.sum(df[df.LogR > 0].LogR))/(-(np.sum(df[df.LogR <= 0].LogR))))
    #Days/Period
finaldf=pd.DataFrame(data=arr,columns=['stock','log_r','pos_log','neg_log','pos_periods','ttl_periods','avg_yield','opt_yield_perc'])
finaldf['sharpe']=abs(finaldf.pos_log/finaldf.neg_log)
finaldf['winr']=finaldf.pos_periods/finaldf.ttl_periods
finaldf.to_csv('big.csv')
print(finaldf)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********